#### Paphos Weather

This page shows various metrics for the weather in Paphos, Cyprus.

Weather data is purchased from Openweather Marketplace [here](https://home.openweathermap.org/marketplace/my_orders)

Download this weather data from here https://drive.google.com/file/d/1swgJS4r9f0CqTd0ZXAOnmJmmmKdNJUUA/view?usp=sharing




In [142]:
def calculate_heat_index(temperature, relative_humidity):
    heat_index = (-8.784695 +
                  1.61139411 * temperature +
                  (2.338549 * relative_humidity) +
                  (-0.14611605 * temperature * relative_humidity) +
                  (-0.01230809 * (temperature ** 2)) +
                  (-0.01642482 * (relative_humidity ** 2)) +
                  (0.00221173 * (temperature ** 2) * relative_humidity) +
                  (0.00072546 * temperature * (relative_humidity ** 2)) +
                  (-0.000003582 * (temperature ** 2) * (relative_humidity ** 2)))
    return heat_index

In [1]:

from datetime import date
from datetime import datetime
import time
import pandas as pd

def toDate(epoch_time):
    return datetime.fromtimestamp(epoch_time)  

In [2]:
import pandas as pd

df=pd.read_json('/Users/walkerrowe/Documents/hypatia/pathosWeather.json')

In [3]:
df.columns

Index(['city_name', 'lat', 'lon', 'main', 'wind', 'clouds', 'weather', 'dt',
       'dt_iso', 'timezone', 'rain'],
      dtype='object')

In [4]:
df['dt'].map(toDate)

0        2000-01-01 02:00:00
1        2000-01-01 03:00:00
2        2000-01-01 04:00:00
3        2000-01-01 05:00:00
4        2000-01-01 06:00:00
                 ...        
181459   2020-09-12 22:00:00
181460   2020-09-12 23:00:00
181461   2020-09-13 00:00:00
181462   2020-09-13 01:00:00
181463   2020-09-13 02:00:00
Name: dt, Length: 181464, dtype: datetime64[ns]

We have to convert the epoch time to ISO Date, which is something we can read.

The map function runs a function over a series.  A series means a dataframe with one column.  When we write df['new column'] = df['some column'].map(somefunction) it adds the new columns to the dataframe.


In [5]:
df['dateTime']=df['dt'].map(lambda l: toDate(l).strftime("%Y-%m-%d %H:%M:%S"))

In [89]:
df['month']=df['dt'].map(lambda l: toDate(l).strftime("%m"))
df['day']=df['dt'].map(lambda l: toDate(l).strftime("%d"))
df['year']=df['dt'].map(lambda l: toDate(l).strftime("%y"))
df['hour']=df['dt'].map(lambda l: toDate(l).strftime("%H"))
df['hour']=pd.to_numeric(df['hour'])

In [140]:
df['temp']=df["main"].map(lambda x: (x["temp"]-32)*(5/9))
df['temp_min']=df["main"].map(lambda x: (x["temp_min"]-32)*(5/9))
df['temp_max']=df["main"].map(lambda x: (x["temp_max"]-32)*(5/9))
df['humidity']=df["main"].map(lambda x: x["humidity"])

In [146]:
df['heatIndex'] = df.apply(lambda row: calculate_heat_index(row['temp'], row['humidity']), axis=1)


In [148]:

summer=df.loc[df['month'].isin(['06','07','08'])]
afternoon=df.loc[(df['hour'] > 13)]

In [153]:
hot=afternoon[['year','month','day', 'hour' ,'temp','temp_min','temp_max','hour', 'humidity', 'heatIndex']]

We want to caculate the average temperature by date usign the mean() function.  But the problem is that the date is an object inside the **main** object.  See below how to fix that.

In [154]:
hot.sort_values(by='temp',ascending=False).head(50)

,year,month,day,hour,temp,temp_min,temp_max,hour,humidity,heatIndex
178644,20,05,18,15,35.777778,27.561111,40.000000,15,30,35.822245
181259,20,09,04,14,35.127778,30.650000,38.450000,14,79,56.449251
178645,20,05,18,16,35.088889,27.561111,38.061111,16,28,34.395851
178643,20,05,18,14,35.050000,27.561111,38.888889,14,36,36.173117
181260,20,09,04,15,34.972222,30.650000,38.450000,15,70,50.238187
178646,20,05,18,17,34.627778,29.044444,37.222222,17,28,33.772205
109907,12,07,15,14,34.538889,28.544444,37.600000,14,36,35.328413
178595,20,05,16,14,34.488889,26.683333,37.222222,14,36,35.247538
181261,20,09,04,16,34.488889,30.650000,38.450000,16,70,48.489502
109908,12,07,15,15,34.477778,28.511111,36.600000,15,36,35.229608
